Import libraries

In [23]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D,Convolution2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.optimizers import SGD, RMSprop, Adam

import matplotlib.pyplot as plt
 
%matplotlib inline

Define variables

In [24]:
# Train data
train_dir = 'classification_data/train'
# Validation data
val_dir = 'classification_data/val'
# Test data
test_dir = 'classification_data/test'
# Image size
img_width, img_height = 521, 521
# Tensor dimensions
input_shape = (img_width, img_height, 3)
# Epoches
epochs = 30
# Batch size
batch_size = 16
# Train samples
nb_train_samples = 16164
# Validation samples
nb_validation_samples = 3000
# Test samples
nb_test_samples = 3000

In [25]:
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))


In [26]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
datagen = ImageDataGenerator(rescale=1. / 255)


In [28]:

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 16164 images belonging to 2 classes.


In [29]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3000 images belonging to 2 classes.


In [30]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3000 images belonging to 2 classes.


In [31]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
   1/1010 [..............................] - ETA: 1:23:41 - loss: 0.7023 - acc: 0.3125

KeyboardInterrupt: 

In [32]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print("Test data accuracy: %.2f%%" % (scores[1]*100))

KeyboardInterrupt: 

In [33]:
def plot_history(history):
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(20,7))
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_title('model accuracy')
    ax1.set_ylabel('accuracy')
    ax1.set_xlabel('epoch')
    ax1.legend(['train', 'test'], loc='upper left')
    
    
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('model loss')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['train', 'test'], loc='upper left')

In [34]:
plot_history(history)

NameError: name 'history' is not defined